In [2]:
import numpy as np
from torchvision.datasets import MNIST
import time
from sklearn.metrics import accuracy_score

def download_mnist(is_train: bool):
    dataset = MNIST(root='./data',
                   transform=lambda x: np.array(x).flatten() / 255.0,
                   download=True,
                   train=is_train)
    mnist_data = []
    mnist_labels = []
    for image, label in dataset:
        mnist_data.append(image)
        mnist_labels.append(label)
    return np.array(mnist_data), np.array(mnist_labels)

# Download and prepare the data
train_X, train_Y = download_mnist(True)


In [3]:

test_X, test_Y = download_mnist(False)

# Convert labels to one-hot encoding
def create_one_hot(labels, num_classes=10):
    one_hot = np.zeros((len(labels), num_classes))
    one_hot[np.arange(len(labels)), labels] = 1
    return one_hot

train_Y = create_one_hot(train_Y)
test_Y = create_one_hot(test_Y)

In [10]:

class MLPClassifier:
    def __init__(
        self,
        input_size: int = 784,
        hidden_size: int = 100,
        output_size: int = 10,
        learning_rate: float = 0.1,
        batch_size: int = 128
    ):
        # Initialize weights using He initialization
        self.W1 = np.random.randn(input_size, hidden_size) * np.sqrt(2.0/input_size)
        self.b1 = np.zeros(hidden_size)
        self.W2 = np.random.randn(hidden_size, output_size) * np.sqrt(2.0/hidden_size)
        self.b2 = np.zeros(output_size)
        
        self.learning_rate = learning_rate
        self.batch_size = batch_size

    def relu(self, x):
        return np.maximum(0, x)

    def relu_derivative(self, x):
        return np.where(x > 0, 1, 0)

    def softmax(self, x):
        exp_x = np.exp(x - np.max(x, axis=1, keepdims=True))
        return exp_x / np.sum(exp_x, axis=1, keepdims=True)

    def forward(self, X, training: bool = True):
        # First layer
        Z1 = np.dot(X, self.W1) + self.b1
        A1 = self.relu(Z1)
        
        # Output layer
        R2 = np.dot(A1, self.W2) + self.b2
        A2 = self.softmax(R2)
        
        return Z1, A1, A2

    def backward(self, X, y, Z1, A1, A2):
        m = X.shape[0]
        
        # Output layer gradients
        dZ2 = A2 - y # derivative of loss with respect to pre-activation
        dW2 = (1/m) * np.dot(A1.T, dZ2)
        db2 = (1/m) * np.sum(dZ2, axis=0)
        
        # Hidden layer gradients
        dA1 = np.dot(dZ2, self.W2.T)
        dZ1 = dA1 * self.relu_derivative(Z1)
        dW1 = (1/m) * np.dot(X.T, dZ1)
        db1 = (1/m) * np.sum(dZ1, axis=0)
        
        # Update weights with L2 regularization
        lambda_reg = 0.0001  # L2 regularization parameter
        self.W2 -= self.learning_rate * (dW2 + lambda_reg * self.W2) # add the gradient of the L2
        self.b2 -= self.learning_rate * db2
        self.W1 -= self.learning_rate * (dW1 + lambda_reg * self.W1)
        self.b1 -= self.learning_rate * db1

    def train(self, X_train, y_train, X_val, y_val, epochs: int = 10):
        start_time = time.time()
        history = []
        
        n_samples = X_train.shape[0]
        
        for epoch in range(epochs):
            
            # Mini-batch training
            for i in range(0, n_samples, self.batch_size):
                batch_X = X_train[i:i + self.batch_size]
                batch_y = y_train[i:i + self.batch_size]
                
                # Forward pass
                Z1, A1, A2 = self.forward(batch_X, training=True)
                
                # Backward pass
                self.backward(batch_X, batch_y, Z1, A1, A2)
            
            # Compute training metrics
            _, _, train_predictions = self.forward(X_train, training=False)
            train_accuracy = accuracy_score(np.argmax(y_train, axis=1), 
                                         np.argmax(train_predictions, axis=1))
            
            # Compute validation metrics
            _, _, val_predictions = self.forward(X_val, training=False)
            val_accuracy = accuracy_score(np.argmax(y_val, axis=1), 
                                       np.argmax(val_predictions, axis=1))
            
            history.append((train_accuracy, val_accuracy))
            
            print(f"Epoch {epoch+1}/{epochs}")
            print(f"Training Accuracy: {train_accuracy:.4f}")
            print(f"Validation Accuracy: {val_accuracy:.4f}")
            print(f"Time elapsed: {time.time() - start_time:.2f}s")
            if history[(-2) % len(history)][0] > train_accuracy and len(history > 50):
                print(f"No further improvement in train accuracy. Stopping training")
                break
            print("-" * 50)
            
        return history

    def predict(self, X):
        _, _, predictions = self.forward(X, training=False)
        return predictions


# Initialize and train the model
mlp = MLPClassifier(
    input_size=784,
    hidden_size=100,
    output_size=10,
    learning_rate=0.1,
    batch_size=128
)

# Train the model
history = mlp.train(train_X, train_Y, test_X, test_Y, epochs=50)

# Make predictions
final_predictions = mlp.predict(test_X)
final_accuracy = accuracy_score(np.argmax(test_Y, axis=1), 
                              np.argmax(final_predictions, axis=1))
print(f"Final Test Accuracy: {final_accuracy:.4f}")

Epoch 1/50
Training Accuracy: 0.9131
Validation Accuracy: 0.9169
Time elapsed: 1.78s
--------------------------------------------------
Epoch 2/50
Training Accuracy: 0.9326
Validation Accuracy: 0.9331
Time elapsed: 3.03s
--------------------------------------------------
Epoch 3/50
Training Accuracy: 0.9438
Validation Accuracy: 0.9408
Time elapsed: 4.43s
--------------------------------------------------
Epoch 4/50
Training Accuracy: 0.9519
Validation Accuracy: 0.9479
Time elapsed: 5.70s
--------------------------------------------------
Epoch 5/50
Training Accuracy: 0.9580
Validation Accuracy: 0.9533
Time elapsed: 7.07s
--------------------------------------------------
Epoch 6/50
Training Accuracy: 0.9627
Validation Accuracy: 0.9581
Time elapsed: 8.28s
--------------------------------------------------
Epoch 7/50
Training Accuracy: 0.9666
Validation Accuracy: 0.9616
Time elapsed: 9.56s
--------------------------------------------------
Epoch 8/50
Training Accuracy: 0.9697
Validation 

$$
L = -\sum_i y_i \log(a_i)
$$
$$
L = -\sum_i y_i \log \left( \frac{e^{z_i}}{\sum_j e^{z_j}} \right)
$$
$$
L = -\sum_i y_i \left( \log(e^{z_i}) - \log \left( \sum_j e^{z_j} \right) \right)
$$
$$
L = -\sum_i y_i \left( z_i - \log \sum_j e^{z_j} \right)
$$
$$
L = -\sum_i y_i z_i + \sum_i y_i \log \sum_j e^{z_j}
$$


### The derivative of \( L \) with respect to \( Z2 ) can be derived as follows:
$$
\frac{\partial}{\partial z_k} \left(-\sum_i y_i z_i + \sum_i y_i \log \sum_j e^{z_j} \right) = -y_k + \sum_i y_i \cdot \frac{\partial}{\partial z_k} \log \sum_j e^{z_j}
$$

Use the chain rule, we get:

$$
= -y_k + \sum_i y_i \cdot \frac{1}{\sum_j e^{z_j}} \cdot e^{z_k} = -y_k + \sum_i y_i \cdot a_k
$$

For one-hot encoded labels, this simplifies to:

$$
= a_k - y_k
$$

### The derivative of \( Z2 \) with respect to \( W2 \) can be derived as follows:
$$
Z2 = A1 \cdot W2 + b2 
$$

$$
 \frac{\partial Z2}{\partial W2} = A1 
$$

The derivative of the loss with respect to pre-activation is:
$$
\frac{\partial L}{\partial Z2} = a_k - y_k 
$$

Using the chain rule, the derivative of the loss with respect to \( W2 \) is:
$$
\frac{\partial L}{\partial W2} = \frac{\partial Z2}{\partial W2} \cdot \frac{\partial L}{\partial Z2}
$$

Therefore:
$$
\frac{\partial L}{\partial W2} = A1^T \cdot dZ2 
$$

### The derivative of the cost function \( L \) with respect to \( A1 \) can be derived as follows:

The derivative of the loss with respect to \( Z2 \) (pre-activation) is:
$$ 
\frac{\partial L}{\partial Z2} = a - y 
$$

Using the chain rule, the derivative of the loss with respect to \( A1 \) is:
$$ 
\frac{\partial L}{\partial A1} = \frac{\partial L}{\partial Z2} \cdot \frac{\partial Z2}{\partial A1}
$$

$$ 
Z2 = A1 \cdot W2 + b2 
$$
$$ 
\frac{\partial Z2}{\partial A1} = W2 
$$

Therefore:
$$ \frac{\partial L}{\partial A1} = \frac{\partial L}{\partial Z2} \cdot W2^T $$